In [ ]:
# ref: https://coolpuppy.readthedocs.io/en/latest/Examples/Walkthrough_API.html

In [ ]:
import os
print(os.getcwd())
os.chdir("/data1/sunwenju/projects_on_940xa/2024041703_mESC_HiC_use_pairtools_cooler_pipeline")
print(os.getcwd())

In [ ]:
! find . -name *rep12.merged.1k.zoomify.mcool

# 读取需要用到的文件

In [ ]:
# 读取 mcool 文件
import cooler
resolution = 10_000
# mcool 文件
nora_wt_mcool_file = "./Nora_WT/Nora_mESC_WT_HiC.rep12.merged.1k.zoomify.mcool"
nora_iaa_mcool_file = "./Nora_IAA/Nora_mESC_IAA_HiC.rep12.merged.1k.zoomify.mcool"
ren_wt_mcool_file = "./Ren_WT/Ren_mESC_WT_HiC.rep12.merged.1k.zoomify.mcool"
ren_iaa_mcool_file = "./Ren_IAA/Ren_mESC_IAA_HiC.rep12.merged.1k.zoomify.mcool"
wn_bpk25_mcool_file = "./WN_BPK25/WN_mESC_BPK25_HiC.rep12.merged.1k.zoomify.mcool"
# 生成 r10k cool 互作矩阵对象
nora_wt_r10k_clr = cooler.Cooler(f'{nora_wt_mcool_file}::/resolutions/{resolution}')
nora_iaa_r10k_clr = cooler.Cooler(f'{nora_iaa_mcool_file}::/resolutions/{resolution}')
ren_wt_r10k_clr = cooler.Cooler(f'{ren_wt_mcool_file}::/resolutions/{resolution}')
ren_iaa_r10k_clr = cooler.Cooler(f'{ren_iaa_mcool_file}::/resolutions/{resolution}')
wn_bpk25_r10k_clr = cooler.Cooler(f'{wn_bpk25_mcool_file}::/resolutions/{resolution}')


In [ ]:
# 读取 chrom view 文件
import pandas as pd
mm10_chr_view_df = pd.read_csv("./mm10.chrom.view.tsv", sep="\t", header=0)
print(mm10_chr_view_df.shape)
mm10_chr_view_df.head()


In [ ]:
# 读取 mESC WT d0246 union CTCF summit 文件 
import pandas as pd
mESC_WT_CTCF_summit_df = pd.read_csv("./mESC_WT_d0246_union_CTCF.summit.sorted.bed", sep="\t", header=None)
mESC_WT_CTCF_summit_df.columns = ["chrom", "start", "end", "fold_enrichment"]
print(mESC_WT_CTCF_summit_df.shape)
mESC_WT_CTCF_summit_df.head()


In [ ]:
# 为 summit 添加四分类标签
mESC_WT_CTCF_summit_df['quartile_FC_score'] = pd.qcut(mESC_WT_CTCF_summit_df['fold_enrichment'], 4, labels=False) + 1
print(mESC_WT_CTCF_summit_df.shape)
mESC_WT_CTCF_summit_df.head()

# 计算生成 expected_cis_df

In [ ]:
from cooltools import expected_cis
# Calculate expected interactions for chromosome arms
nora_wt_r10k_expected_cis_df = expected_cis(clr=nora_wt_r10k_clr, view_df=mm10_chr_view_df, ignore_diags=1, nproc=64)
nora_iaa_r10k_expected_cis_df = expected_cis(clr=nora_iaa_r10k_clr, view_df=mm10_chr_view_df, ignore_diags=1, nproc=64)
ren_wt_r10k_expected_cis_df = expected_cis(clr=ren_wt_r10k_clr, view_df=mm10_chr_view_df, ignore_diags=1, nproc=64)
ren_iaa_r10k_expected_cis_df = expected_cis(clr=ren_iaa_r10k_clr, view_df=mm10_chr_view_df, ignore_diags=1, nproc=64)
wn_bpk25_r10k_expected_cis_df = expected_cis(clr=wn_bpk25_r10k_clr, view_df=mm10_chr_view_df, ignore_diags=1, nproc=64)


# CTCF summit 无分组 pileup

## Nora_WT

In [ ]:
from coolpuppy import coolpup
nora_wt_r10k_clr_ctcf_pup = coolpup.pileup(clr=nora_wt_r10k_clr, 
                                            features=mESC_WT_CTCF_summit_df,
                                            features_format='bed', 
                                            view_df=mm10_chr_view_df,
                                            expected_df=nora_wt_r10k_expected_cis_df,
                                            local=True,
                                            flank=300_000,
                                            min_diag=1,
                                            nproc=32)
nora_wt_r10k_clr_ctcf_pup

In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=nora_wt_r10k_clr_ctcf_pup,
            score=True,
            sym=True,
            cmap='coolwarm',
            scale='log',
            height=5,
            font_scale=1.2,
            plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_mESC_CTCF_BS_noClassified.Nora_WT_r10k.pdf")
plt.savefig("coolpuppy_pileup_plot_for_mESC_CTCF_BS_noClassified.Nora_WT_r10k.png")

## Nora_IAA

In [ ]:
from coolpuppy import coolpup
nora_iaa_r10k_clr_ctcf_pup = coolpup.pileup(clr=nora_iaa_r10k_clr, 
                                            features=mESC_WT_CTCF_summit_df,
                                            features_format='bed', 
                                            view_df=mm10_chr_view_df,
                                            expected_df=nora_iaa_r10k_expected_cis_df,
                                            local=True,
                                            flank=300_000,
                                            min_diag=1,
                                            nproc=32)
nora_iaa_r10k_clr_ctcf_pup

In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=nora_iaa_r10k_clr_ctcf_pup,
            score=True,
            sym=True,
            cmap='coolwarm',
            scale='log',
            height=5,
            font_scale=1.2,
            plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_mESC_CTCF_BS_noClassified.Nora_IAA_r10k.pdf")
plt.savefig("coolpuppy_pileup_plot_for_mESC_CTCF_BS_noClassified.Nora_IAA_r10k.png")


## Ren_WT

In [ ]:
from coolpuppy import coolpup
ren_wt_r10k_clr_ctcf_pup = coolpup.pileup(clr=ren_wt_r10k_clr, 
                                            features=mESC_WT_CTCF_summit_df,
                                            features_format='bed', 
                                            view_df=mm10_chr_view_df,
                                            expected_df=ren_wt_r10k_expected_cis_df,
                                            local=True,
                                            flank=300_000,
                                            min_diag=1,
                                            nproc=32)
ren_wt_r10k_clr_ctcf_pup

In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=ren_wt_r10k_clr_ctcf_pup,
            score=True,
            sym=True,
            cmap='coolwarm',
            scale='log',
            height=5,
            font_scale=1.2,
            plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_mESC_CTCF_BS_noClassified.Ren_WT_r10k.pdf")
plt.savefig("coolpuppy_pileup_plot_for_mESC_CTCF_BS_noClassified.Ren_WT_r10k.png")


## Ren_IAA

In [ ]:
from coolpuppy import coolpup
ren_iaa_r10k_clr_ctcf_pup = coolpup.pileup(clr=ren_iaa_r10k_clr, 
                                            features=mESC_WT_CTCF_summit_df,
                                            features_format='bed', 
                                            view_df=mm10_chr_view_df,
                                            expected_df=ren_iaa_r10k_expected_cis_df,
                                            local=True,
                                            flank=300_000,
                                            min_diag=1,
                                            nproc=32)
ren_iaa_r10k_clr_ctcf_pup

In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=ren_iaa_r10k_clr_ctcf_pup,
            score=True,
            sym=True,
            cmap='coolwarm',
            scale='log',
            height=5,
            font_scale=1.2,
            plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_mESC_CTCF_BS_noClassified.Ren_IAA_r10k.pdf")
plt.savefig("coolpuppy_pileup_plot_for_mESC_CTCF_BS_noClassified.Ren_IAA_r10k.png")

## WN_BPK25

In [ ]:
from coolpuppy import coolpup
wn_bpk25_r10k_clr_ctcf_pup = coolpup.pileup(clr=wn_bpk25_r10k_clr, 
                                            features=mESC_WT_CTCF_summit_df,
                                            features_format='bed', 
                                            view_df=mm10_chr_view_df,
                                            expected_df=wn_bpk25_r10k_expected_cis_df,
                                            local=True,
                                            flank=300_000,
                                            min_diag=1,
                                            nproc=32)
wn_bpk25_r10k_clr_ctcf_pup

In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=wn_bpk25_r10k_clr_ctcf_pup,
            score=True,
            sym=True,
            cmap='coolwarm',
            scale='log',
            height=5,
            font_scale=1.2,
            plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_mESC_CTCF_BS_noClassified.WN_BPK25_r10k.pdf")
plt.savefig("coolpuppy_pileup_plot_for_mESC_CTCF_BS_noClassified.WN_BPK25_r10k.png")

## Nora_WT_IAA_WN_BPK25_merge_in_one

In [ ]:
nora_wn_ctcf_pup_merged_df = pd.concat([nora_wt_r10k_clr_ctcf_pup, nora_iaa_r10k_clr_ctcf_pup, wn_bpk25_r10k_clr_ctcf_pup], ignore_index=True)
nora_wn_ctcf_pup_merged_df


In [ ]:
nora_wn_ctcf_pup_merged_df["group"] = ["Nora_WT", "Nora_IAA", "WN_BPK25"]
nora_wn_ctcf_pup_merged_df


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=nora_wn_ctcf_pup_merged_df,
                cols="group",
                col_order=["Nora_WT", "Nora_IAA", "WN_BPK25"],
                colnames=["Nora_WT", "Nora_IAA", "WN_BPK25"],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_mESC_CTCF_BS_noClassified.Nora_WN_r10k.pdf")
plt.savefig("coolpuppy_pileup_plot_for_mESC_CTCF_BS_noClassified.Nora_WN_r10k.png")


## Ren_WT_IAA_WN_BPK25_merge_in_one

In [ ]:
ren_wn_ctcf_pup_merged_df = pd.concat([ren_wt_r10k_clr_ctcf_pup, ren_iaa_r10k_clr_ctcf_pup, wn_bpk25_r10k_clr_ctcf_pup], ignore_index=True)
ren_wn_ctcf_pup_merged_df


In [ ]:
ren_wn_ctcf_pup_merged_df["group"] = ["Ren_WT", "Ren_IAA", "WN_BPK25"]
ren_wn_ctcf_pup_merged_df


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=ren_wn_ctcf_pup_merged_df,
                cols="group",
                col_order=["Ren_WT", "Ren_IAA", "WN_BPK25"],
                colnames=["Ren_WT", "Ren_IAA", "WN_BPK25"],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_mESC_CTCF_BS_noClassified.Ren_WN_r10k.pdf")
plt.savefig("coolpuppy_pileup_plot_for_mESC_CTCF_BS_noClassified.Ren_WN_r10k.png")


## all experiment merge in one

In [ ]:
nora_wn_ren_ctcf_pup_merged_df = pd.concat([nora_wt_r10k_clr_ctcf_pup, nora_iaa_r10k_clr_ctcf_pup, wn_bpk25_r10k_clr_ctcf_pup, ren_iaa_r10k_clr_ctcf_pup, ren_wt_r10k_clr_ctcf_pup], ignore_index=True)
nora_wn_ren_ctcf_pup_merged_df


In [ ]:
nora_wn_ren_ctcf_pup_merged_df["group"] = ["Nora_WT", "Nora_IAA", "WN_BPK25", "Ren_IAA", "Ren_WT"]
nora_wn_ren_ctcf_pup_merged_df

In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=nora_wn_ren_ctcf_pup_merged_df,
                cols="group",
                col_order=["Nora_WT", "Nora_IAA", "WN_BPK25", "Ren_IAA", "Ren_WT"],
                colnames=["Nora_WT", "Nora_IAA", "WN_BPK25", "Ren_IAA", "Ren_WT"],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_mESC_CTCF_BS_noClassified.Nora_WN_Ren_r10k.pdf")
plt.savefig("coolpuppy_pileup_plot_for_mESC_CTCF_BS_noClassified.Nora_WN_Ren_r10k.png")


## Nora_IAA_vs_WT

In [ ]:
# nora_wt_r10k_clr_ctcf_pup
# nora_iaa_r10k_clr_ctcf_pup
from coolpuppy.lib.puputils import divide_pups
ctcf_pup_nora_iaa_vs_wt = divide_pups(nora_iaa_r10k_clr_ctcf_pup, nora_wt_r10k_clr_ctcf_pup)
ctcf_pup_nora_iaa_vs_wt

In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=ctcf_pup_nora_iaa_vs_wt,
            score=True,
            sym=True,
            cmap='coolwarm',
            scale='log',
            height=5,
            font_scale=1.2,
            plot_ticks=True)
plt.savefig("coolpuppy_pileup_divide_plot_for_mESC_CTCF_BS_noClassified.r10k.Nora_IAA_vs_WT.pdf")
plt.savefig("coolpuppy_pileup_divide_plot_for_mESC_CTCF_BS_noClassified.r10k.Nora_IAA_vs_WT.png")

## Ren_IAA_vs_WT

In [ ]:
# ren_wt_r10k_clr_ctcf_pup
# ren_iaa_r10k_clr_ctcf_pup
from coolpuppy.lib.puputils import divide_pups
ctcf_pup_ren_iaa_vs_wt = divide_pups(ren_iaa_r10k_clr_ctcf_pup, ren_wt_r10k_clr_ctcf_pup)
ctcf_pup_ren_iaa_vs_wt

In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=ctcf_pup_ren_iaa_vs_wt,
            score=True,
            sym=True,
            cmap='coolwarm',
            scale='log',
            height=5,
            font_scale=1.2,
            plot_ticks=True)
plt.savefig("coolpuppy_pileup_divide_plot_for_mESC_CTCF_BS_noClassified.r10k.Ren_IAA_vs_WT.pdf")
plt.savefig("coolpuppy_pileup_divide_plot_for_mESC_CTCF_BS_noClassified.r10k.Ren_IAA_vs_WT.png")

## WN_BPK25_vs_Nora_WT

In [ ]:
# nora_wt_r10k_clr_ctcf_pup
# wn_bpk25_r10k_clr_ctcf_pup
from coolpuppy.lib.puputils import divide_pups
ctcf_pup_wn_bpk25_vs_nora_wt = divide_pups(wn_bpk25_r10k_clr_ctcf_pup, nora_wt_r10k_clr_ctcf_pup)
ctcf_pup_wn_bpk25_vs_nora_wt

In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=ctcf_pup_wn_bpk25_vs_nora_wt,
            score=True,
            sym=True,
            cmap='coolwarm',
            scale='log',
            height=5,
            font_scale=1.2,
            plot_ticks=True)
plt.savefig("coolpuppy_pileup_divide_plot_for_mESC_CTCF_BS_noClassified.r10k.WN_BPK25_vs_Nora_WT.pdf")
plt.savefig("coolpuppy_pileup_divide_plot_for_mESC_CTCF_BS_noClassified.r10k.WN_BPK25_vs_Nora_WT.png")

## WN_BPK25_vs_Ren_WT

In [ ]:
# ren_wt_r10k_clr_ctcf_pup
# wn_bpk25_r10k_clr_ctcf_pup
from coolpuppy.lib.puputils import divide_pups
ctcf_pup_wn_bpk25_vs_ren_wt = divide_pups(wn_bpk25_r10k_clr_ctcf_pup, ren_wt_r10k_clr_ctcf_pup)
ctcf_pup_wn_bpk25_vs_ren_wt

In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=ctcf_pup_wn_bpk25_vs_ren_wt,
            score=True,
            sym=True,
            cmap='coolwarm',
            scale='log',
            height=5,
            font_scale=1.2,
            plot_ticks=True)
plt.savefig("coolpuppy_pileup_divide_plot_for_mESC_CTCF_BS_noClassified.r10k.WN_BPK25_vs_Ren_WT.pdf")
plt.savefig("coolpuppy_pileup_divide_plot_for_mESC_CTCF_BS_noClassified.r10k.WN_BPK25_vs_Ren_WT.png")


## merge 4 divide pileup in one

In [ ]:
ctcf_pup_divide_merged_df = pd.concat([ctcf_pup_nora_iaa_vs_wt, ctcf_pup_ren_iaa_vs_wt, ctcf_pup_wn_bpk25_vs_nora_wt, ctcf_pup_wn_bpk25_vs_ren_wt], ignore_index=True)
ctcf_pup_divide_merged_df


In [ ]:
ctcf_pup_divide_merged_df.insert(0, "IAA_vs_WT", ["Yes", "Yes", "No", "No"] )
ctcf_pup_divide_merged_df.insert(1, "WT_source", ["Nora", "Ren", "Nora", "Ren"] )
ctcf_pup_divide_merged_df

In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=ctcf_pup_divide_merged_df,
                rows="IAA_vs_WT",
                row_order=["Yes", "No"],
                rownames=["IAA_vs_WT", "BPK25_vs_WT"],
                cols='WT_source',
                col_order=["Nora", "Ren"],
                colnames=["use_Nora_WT", "use_Ren_WT"],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_divide_plot_for_mESC_CTCF_BS_noClassified.r10k.merge_4comparation.pdf")
plt.savefig("coolpuppy_pileup_divide_plot_for_mESC_CTCF_BS_noClassified.r10k.merge_4comparation.png")


# CTCF summit FC 四分位 pileup

In [ ]:
mESC_WT_CTCF_summit_df.quartile_FC_score.value_counts()

## Nora_WT

In [ ]:
from coolpuppy import coolpup
nora_wt_r10k_clr_ctcf_fc4Q_pup = coolpup.pileup(clr=nora_wt_r10k_clr, 
                                            features=mESC_WT_CTCF_summit_df,
                                            features_format='bed', 
                                            view_df=mm10_chr_view_df,
                                            expected_df=nora_wt_r10k_expected_cis_df,
                                            groupby=["quartile_FC_score1"],
                                            local=True,
                                            flank=300_000,
                                            min_diag=1,
                                            nproc=32)
nora_wt_r10k_clr_ctcf_fc4Q_pup

In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=nora_wt_r10k_clr_ctcf_fc4Q_pup,
            cols='quartile_FC_score1',
            col_order=["all", 1, 2, 3, 4],
            colnames=["CTCF_ps_ALL", "CTCF_ps_FC_Q1", "CTCF_ps_FC_Q2", "CTCF_ps_FC_Q3", "CTCF_ps_FC_Q4"],
            score=True,
            sym=True,
            cmap='coolwarm',
            scale='log',
            height=5,
            font_scale=1.2,
            plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_mESC_CTCF_BS_FC4Q.Nora_WT_r10k.pdf")
plt.savefig("coolpuppy_pileup_plot_for_mESC_CTCF_BS_FC4Q.Nora_WT_r10k.png")


## Nora_IAA

In [ ]:
from coolpuppy import coolpup
nora_iaa_r10k_clr_ctcf_fc4Q_pup = coolpup.pileup(clr=nora_iaa_r10k_clr, 
                                            features=mESC_WT_CTCF_summit_df,
                                            features_format='bed', 
                                            view_df=mm10_chr_view_df,
                                            expected_df=nora_iaa_r10k_expected_cis_df,
                                            groupby=["quartile_FC_score1"],
                                            local=True,
                                            flank=300_000,
                                            min_diag=1,
                                            nproc=32)
nora_iaa_r10k_clr_ctcf_fc4Q_pup

In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=nora_iaa_r10k_clr_ctcf_fc4Q_pup,
            cols='quartile_FC_score1',
            col_order=["all", 1, 2, 3, 4],
            colnames=["CTCF_bs_ALL", "CTCF_bs_FC_Q1", "CTCF_bs_FC_Q2", "CTCF_bs_FC_Q3", "CTCF_bs_FC_Q4"],
            score=True,
            sym=True,
            cmap='coolwarm',
            scale='log',
            height=5,
            font_scale=1.2,
            plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_mESC_CTCF_BS_FC4Q.Nora_IAA_r10k.pdf")
plt.savefig("coolpuppy_pileup_plot_for_mESC_CTCF_BS_FC4Q.Nora_IAA_r10k.png")

## Ren_WT

In [ ]:
from coolpuppy import coolpup
ren_wt_r10k_clr_ctcf_fc4Q_pup = coolpup.pileup(clr=ren_wt_r10k_clr, 
                                            features=mESC_WT_CTCF_summit_df,
                                            features_format='bed', 
                                            view_df=mm10_chr_view_df,
                                            expected_df=ren_wt_r10k_expected_cis_df,
                                            groupby=["quartile_FC_score1"],
                                            local=True,
                                            flank=300_000,
                                            min_diag=1,
                                            nproc=32)
ren_wt_r10k_clr_ctcf_fc4Q_pup

In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=ren_wt_r10k_clr_ctcf_fc4Q_pup,
            cols='quartile_FC_score1',
            col_order=["all", 1, 2, 3, 4],
            colnames=["CTCF_ps_ALL", "CTCF_ps_FC_Q1", "CTCF_ps_FC_Q2", "CTCF_ps_FC_Q3", "CTCF_ps_FC_Q4"],
            score=True,
            sym=True,
            cmap='coolwarm',
            scale='log',
            height=5,
            font_scale=1.2,
            plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_mESC_CTCF_BS_FC4Q.Ren_WT_r10k.pdf")
plt.savefig("coolpuppy_pileup_plot_for_mESC_CTCF_BS_FC4Q.Ren_WT_r10k.png")


## Ren_IAA

In [ ]:
from coolpuppy import coolpup
ren_iaa_r10k_clr_ctcf_fc4Q_pup = coolpup.pileup(clr=ren_iaa_r10k_clr, 
                                            features=mESC_WT_CTCF_summit_df,
                                            features_format='bed', 
                                            view_df=mm10_chr_view_df,
                                            expected_df=ren_iaa_r10k_expected_cis_df,
                                            groupby=["quartile_FC_score1"],
                                            local=True,
                                            flank=300_000,
                                            min_diag=1,
                                            nproc=32)
ren_iaa_r10k_clr_ctcf_fc4Q_pup


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=ren_iaa_r10k_clr_ctcf_fc4Q_pup,
            cols='quartile_FC_score1',
            col_order=["all", 1, 2, 3, 4],
            colnames=["CTCF_bs_ALL", "CTCF_bs_FC_Q1", "CTCF_bs_FC_Q2", "CTCF_bs_FC_Q3", "CTCF_bs_FC_Q4"],
            score=True,
            sym=True,
            cmap='coolwarm',
            scale='log',
            height=5,
            font_scale=1.2,
            plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_mESC_CTCF_BS_FC4Q.Ren_IAA_r10k.pdf")
plt.savefig("coolpuppy_pileup_plot_for_mESC_CTCF_BS_FC4Q.Ren_IAA_r10k.png")


## WN_BPK25

In [ ]:
from coolpuppy import coolpup
wn_bpk25_r10k_clr_ctcf_fc4Q_pup = coolpup.pileup(clr=wn_bpk25_r10k_clr, 
                                            features=mESC_WT_CTCF_summit_df,
                                            features_format='bed', 
                                            view_df=mm10_chr_view_df,
                                            expected_df=wn_bpk25_r10k_expected_cis_df,
                                            groupby=["quartile_FC_score1"],
                                            local=True,
                                            flank=300_000,
                                            min_diag=1,
                                            nproc=32)
wn_bpk25_r10k_clr_ctcf_fc4Q_pup


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=wn_bpk25_r10k_clr_ctcf_fc4Q_pup,
            cols='quartile_FC_score1',
            col_order=["all", 1, 2, 3, 4],
            colnames=["CTCF_bs_ALL", "CTCF_bs_FC_Q1", "CTCF_bs_FC_Q2", "CTCF_bs_FC_Q3", "CTCF_bs_FC_Q4"],
            score=True,
            sym=True,
            cmap='coolwarm',
            scale='log',
            height=5,
            font_scale=1.2,
            plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_mESC_CTCF_BS_FC4Q.WN_BPK25_r10k.pdf")
plt.savefig("coolpuppy_pileup_plot_for_mESC_CTCF_BS_FC4Q.WN_BPK25_r10k.png")

## Nora_WT_IAA_WN_BPK25_merge_in_one

In [ ]:
nora_wn_ctcf_fc4Q_pup_merged_df = pd.concat([nora_wt_r10k_clr_ctcf_fc4Q_pup, nora_iaa_r10k_clr_ctcf_fc4Q_pup, wn_bpk25_r10k_clr_ctcf_fc4Q_pup], ignore_index=True)
nora_wn_ctcf_fc4Q_pup_merged_df

In [ ]:
nora_wn_ctcf_fc4Q_pup_merged_df["group"] = ['Nora_WT']*5 + ['Nora_IAA']*5 + ['WN_BPK25']*5
nora_wn_ctcf_fc4Q_pup_merged_df

In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=nora_wn_ctcf_fc4Q_pup_merged_df,
                rows="group",
                row_order=["Nora_WT", "Nora_IAA", "WN_BPK25"],
                rownames=["Nora_WT", "Nora_IAA", "WN_BPK25"],
                cols='quartile_FC_score1',
                col_order=["all", 1, 2, 3, 4],
                colnames=["CTCF_ps_ALL", "CTCF_ps_FC_Q1", "CTCF_ps_FC_Q2", "CTCF_ps_FC_Q3", "CTCF_ps_FC_Q4"],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_mESC_CTCF_BS_FC4Q.Nora_WN_r10k.exp_row.pdf")
plt.savefig("coolpuppy_pileup_plot_for_mESC_CTCF_BS_FC4Q.Nora_WN_r10k.exp_row.png")


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=nora_wn_ctcf_fc4Q_pup_merged_df,
                cols="group",
                col_order=["Nora_WT", "Nora_IAA", "WN_BPK25"],
                colnames=["Nora_WT", "Nora_IAA", "WN_BPK25"],
                rows='quartile_FC_score1',
                row_order=["all", 4, 3, 2, 1],
                rownames=["CTCF_ps_ALL", "CTCF_ps_FC_Q4", "CTCF_ps_FC_Q3", "CTCF_ps_FC_Q2", "CTCF_ps_FC_Q1"],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=4,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_mESC_CTCF_BS_FC4Q.Nora_WN_r10k.exp_col.pdf")
plt.savefig("coolpuppy_pileup_plot_for_mESC_CTCF_BS_FC4Q.Nora_WN_r10k.exp_col.png")


## Ren_WT_IAA_WN_BPK25_merge_in_one

In [ ]:
ren_wn_ctcf_fc4Q_pup_merged_df = pd.concat([ren_wt_r10k_clr_ctcf_fc4Q_pup, ren_iaa_r10k_clr_ctcf_fc4Q_pup, wn_bpk25_r10k_clr_ctcf_fc4Q_pup], ignore_index=True)
ren_wn_ctcf_fc4Q_pup_merged_df


In [ ]:
ren_wn_ctcf_fc4Q_pup_merged_df["group"] = ['Ren_WT']*5 + ['Ren_IAA']*5 + ['WN_BPK25']*5
ren_wn_ctcf_fc4Q_pup_merged_df


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=ren_wn_ctcf_fc4Q_pup_merged_df,
                rows="group",
                row_order=["Ren_WT", "Ren_IAA", "WN_BPK25"],
                rownames=["Ren_WT", "Ren_IAA", "WN_BPK25"],
                cols='quartile_FC_score1',
                col_order=["all", 1, 2, 3, 4],
                colnames=["CTCF_ps_ALL", "CTCF_ps_FC_Q1", "CTCF_ps_FC_Q2", "CTCF_ps_FC_Q3", "CTCF_ps_FC_Q4"],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_mESC_CTCF_BS_FC4Q.Ren_WN_r10k.exp_row.pdf")
plt.savefig("coolpuppy_pileup_plot_for_mESC_CTCF_BS_FC4Q.Ren_WN_r10k.exp_row.png")


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=ren_wn_ctcf_fc4Q_pup_merged_df,
                cols="group",
                col_order=["Ren_WT", "Ren_IAA", "WN_BPK25"],
                colnames=["Ren_WT", "Ren_IAA", "WN_BPK25"],
                rows='quartile_FC_score1',
                row_order=["all", 4, 3, 2, 1],
                rownames=["CTCF_ps_ALL", "CTCF_ps_FC_Q4", "CTCF_ps_FC_Q3", "CTCF_ps_FC_Q2", "CTCF_ps_FC_Q1"],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=4,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_mESC_CTCF_BS_FC4Q.Ren_WN_r10k.exp_col.pdf")
plt.savefig("coolpuppy_pileup_plot_for_mESC_CTCF_BS_FC4Q.Ren_WN_r10k.exp_col.png")


## all experiment merge in one

In [ ]:
nora_wn_ren_ctcf_fc4Q_pup_merged_df = pd.concat([nora_wt_r10k_clr_ctcf_fc4Q_pup, nora_iaa_r10k_clr_ctcf_fc4Q_pup, wn_bpk25_r10k_clr_ctcf_fc4Q_pup, ren_iaa_r10k_clr_ctcf_fc4Q_pup, ren_wt_r10k_clr_ctcf_fc4Q_pup], ignore_index=True)
nora_wn_ren_ctcf_fc4Q_pup_merged_df


In [ ]:
nora_wn_ren_ctcf_fc4Q_pup_merged_df["group"] = ['Nora_WT']*5 + ['Nora_IAA']*5 + ['WN_BPK25']*5 + ['Ren_IAA']*5 + ['Ren_WT']*5
nora_wn_ren_ctcf_fc4Q_pup_merged_df


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=nora_wn_ren_ctcf_fc4Q_pup_merged_df,
                rows="group",
                row_order=["Nora_WT", "Nora_IAA", "WN_BPK25", "Ren_IAA", "Ren_WT"],
                rownames=["Nora_WT", "Nora_IAA", "WN_BPK25", "Ren_IAA", "Ren_WT"],
                cols='quartile_FC_score1',
                col_order=["all", 1, 2, 3, 4],
                colnames=["CTCF_ps_ALL", "CTCF_ps_FC_Q1", "CTCF_ps_FC_Q2", "CTCF_ps_FC_Q3", "CTCF_ps_FC_Q4"],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_mESC_CTCF_BS_FC4Q.Nora_WN_Ren_r10k.exp_row.pdf")
plt.savefig("coolpuppy_pileup_plot_for_mESC_CTCF_BS_FC4Q.Nora_WN_Ren_r10k.exp_row.png")


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=nora_wn_ren_ctcf_fc4Q_pup_merged_df,
                cols="group",
                col_order=["Nora_WT", "Nora_IAA", "WN_BPK25", "Ren_IAA", "Ren_WT"],
                colnames=["Nora_WT", "Nora_IAA", "WN_BPK25", "Ren_IAA", "Ren_WT"],
                rows='quartile_FC_score1',
                row_order=["all", 4, 3, 2, 1],
                rownames=["CTCF_ps_ALL", "CTCF_ps_FC_Q4", "CTCF_ps_FC_Q3", "CTCF_ps_FC_Q2", "CTCF_ps_FC_Q1"],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=4,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_mESC_CTCF_BS_FC4Q.Nora_WN_Ren_r10k.exp_col.pdf")
plt.savefig("coolpuppy_pileup_plot_for_mESC_CTCF_BS_FC4Q.Nora_WN_Ren_r10k.exp_col.png")


## Nora_IAA_vs_WT

In [ ]:
# nora_wt_r10k_clr_ctcf_fc4Q_pup
# nora_iaa_r10k_clr_ctcf_fc4Q_pup
from coolpuppy.lib.puputils import divide_pups
all_quartile_group_divide_pup = []
for quartile_group in ["all", 1, 2, 3, 4]:
    iaa_pup = nora_iaa_r10k_clr_ctcf_fc4Q_pup[nora_iaa_r10k_clr_ctcf_fc4Q_pup.quartile_FC_score1==quartile_group]
    wt_pup = nora_wt_r10k_clr_ctcf_fc4Q_pup[nora_wt_r10k_clr_ctcf_fc4Q_pup.quartile_FC_score1==quartile_group]
    iaa_vs_wt = divide_pups(iaa_pup, wt_pup)
    all_quartile_group_divide_pup.append(iaa_vs_wt)
ctcf_fc4Q_pup_nora_iaa_vs_wt = pd.concat(all_quartile_group_divide_pup, ignore_index=True)
ctcf_fc4Q_pup_nora_iaa_vs_wt


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=ctcf_fc4Q_pup_nora_iaa_vs_wt,
            cols='quartile_FC_score1',
            col_order=["all", 1, 2, 3, 4],
            colnames=["CTCF_ps_ALL", "CTCF_ps_FC_Q1", "CTCF_ps_FC_Q2", "CTCF_ps_FC_Q3", "CTCF_ps_FC_Q4"],
            score=True,
            sym=True,
            cmap='coolwarm',
            scale='log',
            height=5,
            font_scale=1.2,
            plot_ticks=True)
plt.savefig("coolpuppy_pileup_divide_plot_for_mESC_CTCF_BS_FC4Q.r10k.Nora_IAA_vs_WT.pdf")
plt.savefig("coolpuppy_pileup_divide_plot_for_mESC_CTCF_BS_FC4Q.r10k.Nora_IAA_vs_WT.png")


## Ren_IAA_vs_WT

In [ ]:
# ren_wt_r10k_clr_ctcf_fc4Q_pup
# ren_iaa_r10k_clr_ctcf_fc4Q_pup
from coolpuppy.lib.puputils import divide_pups
all_quartile_group_divide_pup = []
for quartile_group in ["all", 1, 2, 3, 4]:
    iaa_pup = ren_iaa_r10k_clr_ctcf_fc4Q_pup[ren_iaa_r10k_clr_ctcf_fc4Q_pup.quartile_FC_score1==quartile_group]
    wt_pup = ren_wt_r10k_clr_ctcf_fc4Q_pup[ren_wt_r10k_clr_ctcf_fc4Q_pup.quartile_FC_score1==quartile_group]
    iaa_vs_wt = divide_pups(iaa_pup, wt_pup)
    all_quartile_group_divide_pup.append(iaa_vs_wt)
ctcf_fc4Q_pup_ren_iaa_vs_wt = pd.concat(all_quartile_group_divide_pup, ignore_index=True)
ctcf_fc4Q_pup_ren_iaa_vs_wt


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=ctcf_fc4Q_pup_ren_iaa_vs_wt,
            cols='quartile_FC_score1',
            col_order=["all", 1, 2, 3, 4],
            colnames=["CTCF_ps_ALL", "CTCF_ps_FC_Q1", "CTCF_ps_FC_Q2", "CTCF_ps_FC_Q3", "CTCF_ps_FC_Q4"],
            score=True,
            sym=True,
            cmap='coolwarm',
            scale='log',
            height=5,
            font_scale=1.2,
            plot_ticks=True)
plt.savefig("coolpuppy_pileup_divide_plot_for_mESC_CTCF_BS_FC4Q.r10k.Ren_IAA_vs_WT.pdf")
plt.savefig("coolpuppy_pileup_divide_plot_for_mESC_CTCF_BS_FC4Q.r10k.Ren_IAA_vs_WT.png")


## WN_BPK25_vs_Nora_WT

In [ ]:
# wn_bpk25_r10k_clr_ctcf_fc4Q_pup
# nora_wt_r10k_clr_ctcf_fc4Q_pup
from coolpuppy.lib.puputils import divide_pups
all_quartile_group_divide_pup = []
for quartile_group in ["all", 1, 2, 3, 4]:
    bpk25_pup = wn_bpk25_r10k_clr_ctcf_fc4Q_pup[wn_bpk25_r10k_clr_ctcf_fc4Q_pup.quartile_FC_score1==quartile_group]
    wt_pup = nora_wt_r10k_clr_ctcf_fc4Q_pup[nora_wt_r10k_clr_ctcf_fc4Q_pup.quartile_FC_score1==quartile_group]
    bpk25_vs_wt = divide_pups(bpk25_pup, wt_pup)
    all_quartile_group_divide_pup.append(bpk25_vs_wt)
ctcf_fc4Q_pup_wn_bpk25_vs_nora_wt = pd.concat(all_quartile_group_divide_pup, ignore_index=True)
ctcf_fc4Q_pup_wn_bpk25_vs_nora_wt


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=ctcf_fc4Q_pup_wn_bpk25_vs_nora_wt,
            cols='quartile_FC_score1',
            col_order=["all", 1, 2, 3, 4],
            colnames=["CTCF_ps_ALL", "CTCF_ps_FC_Q1", "CTCF_ps_FC_Q2", "CTCF_ps_FC_Q3", "CTCF_ps_FC_Q4"],
            score=True,
            sym=True,
            cmap='coolwarm',
            scale='log',
            height=5,
            font_scale=1.2,
            plot_ticks=True)
plt.savefig("coolpuppy_pileup_divide_plot_for_mESC_CTCF_BS_FC4Q.r10k.WN_BPK25_vs_Nora_WT.pdf")
plt.savefig("coolpuppy_pileup_divide_plot_for_mESC_CTCF_BS_FC4Q.r10k.WN_BPK25_vs_Nora_WT.png")


## WN_BPK25_vs_Ren_WT

In [ ]:
# wn_bpk25_r10k_clr_ctcf_fc4Q_pup
# ren_wt_r10k_clr_ctcf_fc4Q_pup
from coolpuppy.lib.puputils import divide_pups
all_quartile_group_divide_pup = []
for quartile_group in ["all", 1, 2, 3, 4]:
    bpk25_pup = wn_bpk25_r10k_clr_ctcf_fc4Q_pup[wn_bpk25_r10k_clr_ctcf_fc4Q_pup.quartile_FC_score1==quartile_group]
    wt_pup = ren_wt_r10k_clr_ctcf_fc4Q_pup[ren_wt_r10k_clr_ctcf_fc4Q_pup.quartile_FC_score1==quartile_group]
    bpk25_vs_wt = divide_pups(bpk25_pup, wt_pup)
    all_quartile_group_divide_pup.append(bpk25_vs_wt)
ctcf_fc4Q_pup_wn_bpk25_vs_ren_wt = pd.concat(all_quartile_group_divide_pup, ignore_index=True)
ctcf_fc4Q_pup_wn_bpk25_vs_ren_wt


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=ctcf_fc4Q_pup_wn_bpk25_vs_ren_wt,
            cols='quartile_FC_score1',
            col_order=["all", 1, 2, 3, 4],
            colnames=["CTCF_ps_ALL", "CTCF_ps_FC_Q1", "CTCF_ps_FC_Q2", "CTCF_ps_FC_Q3", "CTCF_ps_FC_Q4"],
            score=True,
            sym=True,
            cmap='coolwarm',
            scale='log',
            height=5,
            font_scale=1.2,
            plot_ticks=True)
plt.savefig("coolpuppy_pileup_divide_plot_for_mESC_CTCF_BS_FC4Q.r10k.WN_BPK25_vs_Ren_WT.pdf")
plt.savefig("coolpuppy_pileup_divide_plot_for_mESC_CTCF_BS_FC4Q.r10k.WN_BPK25_vs_Ren_WT.png")


## merge 4 divide pileup in one

In [ ]:
ctcf_fc4Q_pup_divide_merged_df = pd.concat([ctcf_fc4Q_pup_nora_iaa_vs_wt, ctcf_fc4Q_pup_ren_iaa_vs_wt, ctcf_fc4Q_pup_wn_bpk25_vs_nora_wt, ctcf_fc4Q_pup_wn_bpk25_vs_ren_wt], ignore_index=True)
ctcf_fc4Q_pup_divide_merged_df


In [ ]:
ctcf_fc4Q_pup_divide_merged_df.insert(0, "divide_group", ["Nora_IAA_vs_WT"]*5+["Ren_IAA_vs_WT"]*5+["WN_BPK25_vs_Nora_WT"]*5+["WN_BPK25_vs_Ren_WT"]*5 )
ctcf_fc4Q_pup_divide_merged_df


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=ctcf_fc4Q_pup_divide_merged_df,
                rows="divide_group",
                row_order=["Nora_IAA_vs_WT", "Ren_IAA_vs_WT", "WN_BPK25_vs_Nora_WT", "WN_BPK25_vs_Ren_WT"],
                rownames=["Nora_IAA_vs_WT", "Ren_IAA_vs_WT", "WN_BPK25_vs_Nora_WT", "WN_BPK25_vs_Ren_WT"],
                cols='quartile_FC_score1',
                col_order=["all", 1, 2, 3, 4],
                colnames=["CTCF_ps_ALL", "CTCF_ps_FC_Q1", "CTCF_ps_FC_Q2", "CTCF_ps_FC_Q3", "CTCF_ps_FC_Q4"],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_divide_plot_for_mESC_CTCF_BS_FC4Q.r10k.merge_4comparation.exp_row.pdf")
plt.savefig("coolpuppy_pileup_divide_plot_for_mESC_CTCF_BS_FC4Q.r10k.merge_4comparation.exp_row.png")


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=ctcf_fc4Q_pup_divide_merged_df,
                cols="divide_group",
                col_order=["Nora_IAA_vs_WT", "Ren_IAA_vs_WT", "WN_BPK25_vs_Nora_WT", "WN_BPK25_vs_Ren_WT"],
                colnames=["Nora_IAA_vs_WT", "Ren_IAA_vs_WT", "WN_BPK25_vs_Nora_WT", "WN_BPK25_vs_Ren_WT"],
                rows='quartile_FC_score1',
                row_order=["all", 1, 2, 3, 4],
                rownames=["CTCF_ps_ALL", "CTCF_ps_FC_Q1", "CTCF_ps_FC_Q2", "CTCF_ps_FC_Q3", "CTCF_ps_FC_Q4"],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_divide_plot_for_mESC_CTCF_BS_FC4Q.r10k.merge_4comparation.exp_col.pdf")
plt.savefig("coolpuppy_pileup_divide_plot_for_mESC_CTCF_BS_FC4Q.r10k.merge_4comparation.exp_col.png")


# 制作结果汇总 PPT

In [ ]:
! ls -1 coolpuppy_*_for_mESC_CTCF_BS*r10k*.pdf

In [ ]:
! ls -1 coolpuppy_*_for_mESC_CTCF_BS*r10k*.png

In [ ]:
from PIL import Image
from pptx import Presentation
from pptx.util import Pt, Cm
from pptx.dml.color import RGBColor
from pptx.enum.text import PP_ALIGN

CTCF_BS_noC_Nora_WT_r10k_png = "coolpuppy_pileup_plot_for_mESC_CTCF_BS_noClassified.Nora_WT_r10k.png"
CTCF_BS_noC_Nora_IAA_r10k_png = "coolpuppy_pileup_plot_for_mESC_CTCF_BS_noClassified.Nora_IAA_r10k.png"
CTCF_BS_noC_Ren_WT_r10k_png = "coolpuppy_pileup_plot_for_mESC_CTCF_BS_noClassified.Ren_WT_r10k.png"
CTCF_BS_noC_Ren_IAA_r10k_png = "coolpuppy_pileup_plot_for_mESC_CTCF_BS_noClassified.Ren_IAA_r10k.png"
CTCF_BS_noC_WN_BPK25_r10k_png = "coolpuppy_pileup_plot_for_mESC_CTCF_BS_noClassified.WN_BPK25_r10k.png"

CTCF_BS_noC_Nora_WN_r10k_png = "coolpuppy_pileup_plot_for_mESC_CTCF_BS_noClassified.Nora_WN_r10k.png"
CTCF_BS_noC_Ren_WN_r10k_png = "coolpuppy_pileup_plot_for_mESC_CTCF_BS_noClassified.Ren_WN_r10k.png"
CTCF_BS_noC_Nora_WN_Ren_r10k_png = "coolpuppy_pileup_plot_for_mESC_CTCF_BS_noClassified.Nora_WN_Ren_r10k.png"

CTCF_BS_noC_Nora_IAA_vs_WT = "coolpuppy_pileup_divide_plot_for_mESC_CTCF_BS_noClassified.r10k.Nora_IAA_vs_WT.png"
CTCF_BS_noC_Ren_IAA_vs_WT = "coolpuppy_pileup_divide_plot_for_mESC_CTCF_BS_noClassified.r10k.Ren_IAA_vs_WT.png"
CTCF_BS_noC_WN_BPK25_vs_Nora_WT = "coolpuppy_pileup_divide_plot_for_mESC_CTCF_BS_noClassified.r10k.WN_BPK25_vs_Nora_WT.png"
CTCF_BS_noC_WN_BPK25_vs_Ren_WT = "coolpuppy_pileup_divide_plot_for_mESC_CTCF_BS_noClassified.r10k.WN_BPK25_vs_Ren_WT.png"
CTCF_BS_noC_merge_4comparation = "coolpuppy_pileup_divide_plot_for_mESC_CTCF_BS_noClassified.r10k.merge_4comparation.png"

def add_text_anno(slide, anno_text, left, top, width, height):
    # 插入文本框
    text_box = slide.shapes.add_textbox(left, top, width, height)
    # 设置文本框背景颜色为黄色
    text_box.fill.solid()
    text_box.fill.fore_color.rgb = RGBColor(255, 230, 153)
    # 在文本框中添加内容并设置格式
    text_box.text_frame.text = anno_text
    text_box.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
    # 设置字体大小和加粗
    text_font = text_box.text_frame.paragraphs[0].font
    text_font.name = '微软雅黑'
    text_font.size = Pt(6)
    text_font.bold = True


# 创建 PPT 并插入图片
my_ppt = Presentation()
# 设置幻灯片尺寸为 16:9 (25.4cm *14.28cm)
my_ppt.slide_width = 9144000
my_ppt.slide_height = 5143500
# slide1
slide1 = my_ppt.slides.add_slide(my_ppt.slide_layouts[6])
# PPT 页面指定区域插入图片
# pic = slide.shapes.add_picture(png_files[pi-1], left, top, width, height)
pic = slide1.shapes.add_picture(CTCF_BS_noC_Nora_WT_r10k_png, Cm(0.7), Cm(0.05), Cm(7.125), Cm(7.125))
pic = slide1.shapes.add_picture(CTCF_BS_noC_Nora_IAA_r10k_png, Cm(9.14), Cm(0.05), Cm(7.125), Cm(7.125))
pic = slide1.shapes.add_picture(CTCF_BS_noC_WN_BPK25_r10k_png, Cm(17.625), Cm(0.05), Cm(7.125), Cm(7.125))
pic = slide1.shapes.add_picture(CTCF_BS_noC_Ren_WT_r10k_png, Cm(0.7), Cm(7.14), Cm(7.125), Cm(7.125))
pic = slide1.shapes.add_picture(CTCF_BS_noC_Ren_IAA_r10k_png, Cm(9.14), Cm(7.14), Cm(7.125), Cm(7.125))
# 插入文本注释
add_text_anno(slide1, "CTCF_BS_noC_Nora_WT_r10k_pileup_plot", Cm(0.1), Cm(0.05), Cm(8), Cm(0.5))
add_text_anno(slide1, "CTCF_BS_noC_Nora_IAA_r10k_pileup_plot", Cm(8.65), Cm(0.05), Cm(8), Cm(0.5))
add_text_anno(slide1, "CTCF_BS_noC_WN_BPK25_r10k_pileup_plot", Cm(17.3), Cm(0.05), Cm(8), Cm(0.5))
add_text_anno(slide1, "CTCF_BS_noC_Ren_WT_r10k_pileup_plot", Cm(0.1), Cm(7.24), Cm(8), Cm(0.5))
add_text_anno(slide1, "CTCF_BS_noC_Ren_IAA_r10k_pileup_plot", Cm(8.65), Cm(7.24), Cm(8), Cm(0.5))

# slide2
slide2 = my_ppt.slides.add_slide(my_ppt.slide_layouts[6])
# PPT 页面指定区域插入图片
slide2.shapes.add_picture(CTCF_BS_noC_Nora_WN_r10k_png, Cm(6.23), Cm(0.02), Cm(12.94), Cm(4.31))
slide2.shapes.add_picture(CTCF_BS_noC_Ren_WN_r10k_png, Cm(6.23), Cm(4.54),  Cm(12.94), Cm(4.31))
slide2.shapes.add_picture(CTCF_BS_noC_Nora_WN_Ren_r10k_png, Cm(0), Cm(9.04), Cm(25.4), Cm(5.08))
# 插入文本注释
add_text_anno(slide2, "CTCF_BS_noC_Nora_WN_r10k_pileup_plot", Cm(0.28), Cm(1.92), Cm(7), Cm(0.5))
add_text_anno(slide2, "CTCF_BS_noC_Ren_WN_r10k_pileup_plot", Cm(0.28), Cm(6.44), Cm(7), Cm(0.5))
add_text_anno(slide2, "CTCF_BS_noC_Nora_WN_Ren_r10k_pileup_plot", Cm(0.28), Cm(8.86), Cm(7), Cm(0.5))

# slide3
slide3 = my_ppt.slides.add_slide(my_ppt.slide_layouts[6])
# PPT 页面指定区域插入图片
slide3.shapes.add_picture(CTCF_BS_noC_Nora_IAA_vs_WT, Cm(0), Cm(0.8), Cm(6.34), Cm(6.34))
slide3.shapes.add_picture(CTCF_BS_noC_Ren_IAA_vs_WT, Cm(6.34), Cm(0.8),  Cm(6.34), Cm(6.34))
slide3.shapes.add_picture(CTCF_BS_noC_WN_BPK25_vs_Ren_WT, Cm(0), Cm(7.15), Cm(6.34), Cm(6.34))
slide3.shapes.add_picture(CTCF_BS_noC_WN_BPK25_vs_Ren_WT, Cm(6.34), Cm(7.15), Cm(6.34), Cm(6.34))
slide3.shapes.add_picture(CTCF_BS_noC_merge_4comparation, Cm(12.72), Cm(0.8), Cm(12.68), Cm(12.68))
# 插入文本注释
add_text_anno(slide3, "CTCF_BS_noC_Nora_IAA_vs_WT_r10k", Cm(0), Cm(0.68), Cm(6.28), Cm(0.5))
add_text_anno(slide3, "CTCF_BS_noC_Ren_IAA_vs_WT_r10k", Cm(6.43), Cm(0.68), Cm(6.28), Cm(0.5))
add_text_anno(slide3, "CTCF_BS_noC_WN_BPK25_vs_Ren_WT_r10k", Cm(0), Cm(7.27), Cm(6.28), Cm(0.5))
add_text_anno(slide3, "CTCF_BS_noC_WN_BPK25_vs_Ren_WT_r10k", Cm(6.43), Cm(7.27), Cm(6.28), Cm(0.5))
add_text_anno(slide3, "CTCF_BS_noC_merge_4comparation_r10k", Cm(15.86), Cm(0.68), Cm(6.28), Cm(0.5))


# 保存 PPT
my_ppt.save('8.1_coolpuppy_pileup_plot_for_mESC_WT_CTCF_BS.use_Nora_WN_Ren_mcool.r10k.summary.pptx')


In [ ]:
CTCF_BS_FC4Q_Nora_WT_r10k_png = "coolpuppy_pileup_plot_for_mESC_CTCF_BS_FC4Q.Nora_WT_r10k.png"
CTCF_BS_FC4Q_Nora_IAA_r10k_png = "coolpuppy_pileup_plot_for_mESC_CTCF_BS_FC4Q.Nora_IAA_r10k.png"
CTCF_BS_FC4Q_Ren_WT_r10k_png = "coolpuppy_pileup_plot_for_mESC_CTCF_BS_FC4Q.Ren_WT_r10k.png"
CTCF_BS_FC4Q_Ren_IAA_r10k_png = "coolpuppy_pileup_plot_for_mESC_CTCF_BS_FC4Q.Ren_IAA_r10k.png"
CTCF_BS_FC4Q_WN_BPK25_r10k_png = "coolpuppy_pileup_plot_for_mESC_CTCF_BS_FC4Q.WN_BPK25_r10k.png"

# 打开 PPT 并插入图片
my_ppt = Presentation('8.1_coolpuppy_pileup_plot_for_mESC_WT_CTCF_BS.use_Nora_WN_Ren_mcool.r10k.summary.pptx')
# slide4
slide4 = my_ppt.slides.add_slide(my_ppt.slide_layouts[6])
# slide5
slide5 = my_ppt.slides.add_slide(my_ppt.slide_layouts[6])
# PPT 页面指定区域插入图片
# pic = slide.shapes.add_picture(png_files[pi-1], left, top, width, height)
pic = slide5.shapes.add_picture(CTCF_BS_FC4Q_Nora_WT_r10k_png, Cm(1.12), Cm(0.05), Cm(23.17), Cm(4.63))
pic = slide5.shapes.add_picture(CTCF_BS_FC4Q_Nora_IAA_r10k_png, Cm(1.12), Cm(4.83), Cm(23.17), Cm(4.63))
pic = slide5.shapes.add_picture(CTCF_BS_FC4Q_WN_BPK25_r10k_png, Cm(1.12), Cm(9.60), Cm(23.17), Cm(4.63))
# 插入文本注释
add_text_anno(slide5, "CTCF_BS_FC4Q_Nora_WT_r10k_pileup_plot", Cm(0), Cm(0), Cm(8), Cm(0.5))
add_text_anno(slide5, "CTCF_BS_FC4Q_Nora_IAA_r10k_pileup_plot", Cm(0), Cm(4.71), Cm(8), Cm(0.5))
add_text_anno(slide5, "CTCF_BS_FC4Q_WN_BPK25_r10k_pileup_plot", Cm(0), Cm(9.46), Cm(8), Cm(0.5))

# slide6
slide6 = my_ppt.slides.add_slide(my_ppt.slide_layouts[6])
# PPT 页面指定区域插入图片
# pic = slide.shapes.add_picture(png_files[pi-1], left, top, width, height)
pic = slide6.shapes.add_picture(CTCF_BS_FC4Q_Ren_WT_r10k_png, Cm(1.12), Cm(0.05), Cm(23.17), Cm(4.63))
pic = slide6.shapes.add_picture(CTCF_BS_FC4Q_Ren_IAA_r10k_png, Cm(1.12), Cm(4.83), Cm(23.17), Cm(4.63))
pic = slide6.shapes.add_picture(CTCF_BS_FC4Q_WN_BPK25_r10k_png, Cm(1.12), Cm(9.60), Cm(23.17), Cm(4.63))
# 插入文本注释
add_text_anno(slide6, "CTCF_BS_FC4Q_Ren_WT_r10k_pileup_plot", Cm(0), Cm(0), Cm(8), Cm(0.5))
add_text_anno(slide6, "CTCF_BS_FC4Q_Ren_IAA_r10k_pileup_plot", Cm(0), Cm(4.71), Cm(8), Cm(0.5))
add_text_anno(slide6, "CTCF_BS_FC4Q_WN_BPK25_r10k_pileup_plot", Cm(0), Cm(9.46), Cm(8), Cm(0.5))

# 保存 PPT
my_ppt.save('8.1_coolpuppy_pileup_plot_for_mESC_WT_CTCF_BS.use_Nora_WN_Ren_mcool.r10k.summary.pptx')


In [ ]:
CTCF_BS_FC4Q_Nora_WN_r10k_exp_row_png = 'coolpuppy_pileup_plot_for_mESC_CTCF_BS_FC4Q.Nora_WN_r10k.exp_row.png'
CTCF_BS_FC4Q_Ren_WN_r10k_exp_row_png = 'coolpuppy_pileup_plot_for_mESC_CTCF_BS_FC4Q.Ren_WN_r10k.exp_row.png'
CTCF_BS_FC4Q_Nora_WN_r10k_exp_col_png = 'coolpuppy_pileup_plot_for_mESC_CTCF_BS_FC4Q.Nora_WN_r10k.exp_col.png'
CTCF_BS_FC4Q_Ren_WN_r10k_exp_col_png = 'coolpuppy_pileup_plot_for_mESC_CTCF_BS_FC4Q.Ren_WN_r10k.exp_col.png'
CTCF_BS_FC4Q_Nora_WN_Ren_r10k_exp_row_png = 'coolpuppy_pileup_plot_for_mESC_CTCF_BS_FC4Q.Nora_WN_Ren_r10k.exp_row.png'
CTCF_BS_FC4Q_Nora_WN_Ren_r10k_exp_col_png = 'coolpuppy_pileup_plot_for_mESC_CTCF_BS_FC4Q.Nora_WN_Ren_r10k.exp_col.png'

# 打开 PPT 并插入图片
my_ppt = Presentation('8.1_coolpuppy_pileup_plot_for_mESC_WT_CTCF_BS.use_Nora_WN_Ren_mcool.r10k.summary.pptx')
# slide7
slide7 = my_ppt.slides.add_slide(my_ppt.slide_layouts[6])
# PPT 页面指定区域插入图片
pic = slide7.shapes.add_picture(CTCF_BS_FC4Q_Nora_WN_r10k_exp_row_png, Cm(0.79), Cm(0), Cm(23.81), Cm(14.29))
# 插入文本注释
add_text_anno(slide7, "CTCF_BS_FC4Q_Nora_WN_r10k_exp_row_pileup_plot", Cm(0), Cm(0), Cm(8), Cm(0.5))

# slide8
slide8 = my_ppt.slides.add_slide(my_ppt.slide_layouts[6])
# PPT 页面指定区域插入图片
pic = slide8.shapes.add_picture(CTCF_BS_FC4Q_Ren_WN_r10k_exp_row_png, Cm(0.79), Cm(0), Cm(23.81), Cm(14.29))
# 插入文本注释
add_text_anno(slide8, "CTCF_BS_FC4Q_Ren_WN_r10k_exp_row_pileup_plot", Cm(0), Cm(0), Cm(8), Cm(0.5))

# slide9
slide9 = my_ppt.slides.add_slide(my_ppt.slide_layouts[6])
# PPT 页面指定区域插入图片
pic = slide9.shapes.add_picture(CTCF_BS_FC4Q_Nora_WN_r10k_exp_col_png, Cm(2.22), Cm(0), Cm(8.57), Cm(14.29))
pic = slide9.shapes.add_picture(CTCF_BS_FC4Q_Ren_WN_r10k_exp_col_png, Cm(14.62), Cm(0), Cm(8.57), Cm(14.29))
# 插入文本注释
add_text_anno(slide9, "CTCF_BS_FC4Q_Nora_WN_r10k_exp_col_pileup_plot", Cm(0), Cm(0), Cm(8), Cm(0.5))
add_text_anno(slide9, "CTCF_BS_FC4Q_Ren_WN_r10k_exp_col_pileup_plot", Cm(17.4), Cm(0), Cm(8), Cm(0.5))

# slide10
slide10 = my_ppt.slides.add_slide(my_ppt.slide_layouts[6])
# PPT 页面指定区域插入图片
pic = slide10.shapes.add_picture(CTCF_BS_FC4Q_Nora_WN_Ren_r10k_exp_row_png, Cm(0), Cm(0.79), Cm(12.7), Cm(12.7))
pic = slide10.shapes.add_picture(CTCF_BS_FC4Q_Nora_WN_Ren_r10k_exp_col_png, Cm(12.7), Cm(0.79), Cm(12.7), Cm(12.7))
# 插入文本注释
add_text_anno(slide10, "CTCF_BS_FC4Q_Nora_WN_Ren_r10k_exp_row_pileup_plot", Cm(0), Cm(0), Cm(8), Cm(0.5))
add_text_anno(slide10, "CTCF_BS_FC4Q_Nora_WN_Ren_r10k_exp_col_pileup_plot", Cm(17.4), Cm(0), Cm(8), Cm(0.5))

# 保存 PPT
my_ppt.save('8.1_coolpuppy_pileup_plot_for_mESC_WT_CTCF_BS.use_Nora_WN_Ren_mcool.r10k.summary.pptx')


In [ ]:
CTCF_BS_FC4Q_Nora_IAA_vs_WT = "coolpuppy_pileup_divide_plot_for_mESC_CTCF_BS_FC4Q.r10k.Nora_IAA_vs_WT.png"
CTCF_BS_FC4Q_Ren_IAA_vs_WT = "coolpuppy_pileup_divide_plot_for_mESC_CTCF_BS_FC4Q.r10k.Ren_IAA_vs_WT.png"
CTCF_BS_FC4Q_WN_BPK25_vs_Nora_WT = "coolpuppy_pileup_divide_plot_for_mESC_CTCF_BS_FC4Q.r10k.WN_BPK25_vs_Nora_WT.png"
CTCF_BS_FC4Q_WN_BPK25_vs_Ren_WT = "coolpuppy_pileup_divide_plot_for_mESC_CTCF_BS_FC4Q.r10k.WN_BPK25_vs_Ren_WT.png"
CTCF_BS_FC4Q_merge_4comparation_exp_col = "coolpuppy_pileup_divide_plot_for_mESC_CTCF_BS_FC4Q.r10k.merge_4comparation.exp_col.png"
CTCF_BS_FC4Q_merge_4comparation_exp_row = "coolpuppy_pileup_divide_plot_for_mESC_CTCF_BS_FC4Q.r10k.merge_4comparation.exp_row.png"

# 打开 PPT 并插入图片
my_ppt = Presentation('8.1_coolpuppy_pileup_plot_for_mESC_WT_CTCF_BS.use_Nora_WN_Ren_mcool.r10k.summary.pptx')
# slide11
slide11 = my_ppt.slides.add_slide(my_ppt.slide_layouts[6])
# PPT 页面指定区域插入图片
pic = slide11.shapes.add_picture(CTCF_BS_FC4Q_Nora_IAA_vs_WT, Cm(0), Cm(0.6), Cm(13.97), Cm(2.79))
pic = slide11.shapes.add_picture(CTCF_BS_FC4Q_Ren_IAA_vs_WT, Cm(0), Cm(4.11), Cm(13.97), Cm(2.79))
pic = slide11.shapes.add_picture(CTCF_BS_FC4Q_WN_BPK25_vs_Nora_WT, Cm(0), Cm(7.61), Cm(13.97), Cm(2.79))
pic = slide11.shapes.add_picture(CTCF_BS_FC4Q_WN_BPK25_vs_Ren_WT, Cm(0), Cm(11.12), Cm(13.97), Cm(2.79))
pic = slide11.shapes.add_picture(CTCF_BS_FC4Q_merge_4comparation_exp_col, Cm(13.97), Cm(0), Cm(11.43), Cm(14.29))
# 插入文本注释
add_text_anno(slide11, "CTCF_BS_FC4Q_Nora_IAA_vs_WT_pileup_plot", Cm(0), Cm(0), Cm(8), Cm(0.5))
add_text_anno(slide11, "CTCF_BS_FC4Q_Ren_IAA_vs_WT_pileup_plot", Cm(0), Cm(3.61), Cm(8), Cm(0.5))
add_text_anno(slide11, "CTCF_BS_FC4Q_WN_BPK25_vs_Nora_WT_pileup_plot", Cm(0), Cm(7.15), Cm(8), Cm(0.5))
add_text_anno(slide11, "CTCF_BS_FC4Q_WN_BPK25_vs_Ren_WT_pileup_plot", Cm(0), Cm(10.62), Cm(8), Cm(0.5))
add_text_anno(slide11, "CTCF_BS_FC4Q_merge_4comparation_exp_col_pileup_plot", Cm(17.4), Cm(0), Cm(8), Cm(0.5))

# slide12
slide12 = my_ppt.slides.add_slide(my_ppt.slide_layouts[6])
# PPT 页面指定区域插入图片
pic = slide12.shapes.add_picture(CTCF_BS_FC4Q_merge_4comparation_exp_row, Cm(3.75), Cm(0), Cm(17.9), Cm(14.28))
# 插入文本注释
add_text_anno(slide12, "CTCF_BS_FC4Q_merge_4comparation_exp_row_pileup_plot", Cm(0), Cm(0), Cm(8), Cm(0.5))

# 保存 PPT
my_ppt.save('8.1_coolpuppy_pileup_plot_for_mESC_WT_CTCF_BS.use_Nora_WN_Ren_mcool.r10k.summary.pptx')


# 移动相关结果到特定文件夹

In [ ]:
%%bash
mkdir coolpuppy_pileup_plot_for_mESC_CTCF_BS_use_e5_r10k
mv coolpuppy_*_for_mESC_CTCF_BS*r10k*.pdf ./coolpuppy_pileup_plot_for_mESC_CTCF_BS_use_e5_r10k/
mv coolpuppy_*_for_mESC_CTCF_BS*r10k*.png ./coolpuppy_pileup_plot_for_mESC_CTCF_BS_use_e5_r10k/
